Read the message data into 'df' DataFrame, label columns (The given dataset here doesn't exist anymore so replace it with any other dataset)

In [ ]:
import pandas as pd
import numpy as np

df=pd.read_table("https://raw.githubusercontent.com/justmarkham/pycon-2016-tutorial/master/data/sms.tsv",header=None)
names=['label','messages']
df.columns=names

Convert values in 'label' column to numerical values and store in 'label_num' column for using MultinomialNB algorithm that only takes numerical data

In [18]:
df['label_num'] = df.label.map({'ham':0, 'spam':1})

Train-Test Split the message data using sklearn

X_train gives Training message data on which algorithm is trained on (excludes output or 'salary' column)

X_test gives Testing message data on which algorithm is tested on (excludes output or 'salary' column)

y_train gives 'salary' column or output column for prediction derived from training data 

y_train gives 'salary' column or output column to be checked for prediction derived from testing data and calculate accuracy of algorithm

The entire data is split 5 times into training and testing data such that each row of training data is once part of test data
The results of test data after 5 splits is combined according to test_index, i.e index of the rows of original data that came in test data. Then this combined predictions is compared with output values of original data

In [31]:
from sklearn.feature_extraction.text import CountVectorizer

from sklearn.model_selection import KFold
kf = KFold(n_splits=5)

from sklearn.naive_bayes import MultinomialNB
nb = MultinomialNB()

results=pd.Series()

for train_index, test_index in kf.split(df):
    XTrain, XTest, YTrain, ytest = df['messages'].iloc[train_index], df['messages'].iloc[test_index], df['label_num'].iloc[train_index], df['label_num'].iloc[test_index]
    
    cv=CountVectorizer(stop_words=None, token_pattern=r"(?u)\b\w+\b")
    xtrain=cv.fit_transform(XTrain)
    xtest=cv.transform(XTest)
    nb.fit(xtrain,YTrain)
    predict_list = nb.predict(xtest)
    
    indexes=list(test_index)
    
    res=pd.Series(predict_list,index=indexes)
    results=results.append(res)
    
results=results.sort_index()           

In [30]:
from sklearn import metrics
print("Accuracy:",(metrics.accuracy_score(df['label_num'], results)*100),"%")

Accuracy: 98.56424982053123 %
